In [12]:
# importing all libraries needed 

import pandas as pd
import numpy as np
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import xml
import json
from pandas.io.json import json_normalize
import folium
# collecting website
url = requests.get ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

from geopy.geocoders import Nominatim
address = 'Toronto, Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print (latitude,longitude)

table_data = soup.find('table')
fields = table_data.find_all('td')
postcode = []
borough=[]
neighborhood = []
for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighborhood.append(fields[i+2].text.strip())

df_ca = pd.DataFrame(data=[postcode, borough, neighborhood]).transpose()
df_ca.columns = ['Postcode', 'Borough', 'Neighborhood']
df_ca.head(10), df_ca.tail(10)
df_ca['Borough'].replace('Not assigned', np.nan, inplace = True)
df_ca.dropna(subset=['Borough'], inplace =True)
df_ca_new= df_ca.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_ca_new.columns = ['Postcode', 'Borough', 'Neighborhood']
df_ca_new['Neighborhood'].replace('Not assigned', "Queen's Park", inplace =True)

df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns = ['Postcode', 'Latitude', 'Longitude']
df_new = pd.merge(df_ca_new, df_geo, on = ['Postcode'], how = 'inner')
df_arrangement = df_new[[ 'Borough', 'Neighborhood', 'Postcode', 'Latitude', 'Longitude']].copy()
df_arrangement.head(12)
df_downt = df_arrangement[df_arrangement['Borough'].str.contains('Toronto')]
to_data = df_downt.reset_index(drop = True)
map_tohood = folium.Map(location = [latitude, longitude],zoom_start = 20)

to_data.loc[0, 'Neighborhood']


for lat, lng, borough, neighbourhood in zip(to_data['Latitude'], to_data['Longitude'], to_data['Borough'], to_data['Neighborhood']):
            label = '{}, {}'.format(neighborhood, borough)
            label = folium.Popup(label, parse_html=True)
            folium.CircleMarker(
            [lat,lng],
            radius = 3,
            popup = label,
            color = 'green',
            fill = True,
            fill_color = '#3199cc',
            fill_opacity = 0.3,
            parse_html = False).add_to(map_tohood)
map_tohood

to_data

neighborhood_latitude = to_data.loc[0, 'Latitude']
neighborhood_longitude = to_data.loc[0, 'Longitude']
neighborhood_name = to_data.loc[0, 'Neighborhood']
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name,
                                                              neighborhood_latitude,
                                                              neighborhood_longitude))

43.653963 -79.387207
Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


EXPLORING THE NEIGHBORHOOD

In [13]:
from geopy.geocoders import Nominatim
address = 'Toronto, Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print (latitude,longitude)

toronto_map = folium.Map(location =[latitude, longitude], zoom_start = 20)

#adding markers to map
for lat, lng, borough, neighbourhood in zip (df_arrangement['Latitude'], df_arrangement['Longitude'], df_arrangement['Borough'],
                                           df_arrangement['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    redius = 3,
    popup = label,
    color = 'green',
    fill= True,
    fill_color = '#3199cc',
    fill_opacity = 0.3,
    parse_html = False).add_to(toronto_map)
    
toronto_map




43.653963 -79.387207


In [17]:

LIMIT = 100
radius = 500
CLIENT_ID = 'APLB00CYTRCHYGPLBMZVR114O1LX34W0AGQLO4HYPI5CCKQW'
CLIENT_SECRET = 'HVE35SJ45Z0BZSDDGBBWQBSAWQIX55KIRUQABAEMR55T1PNP'
VERSION = '20190413'

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
   CLIENT_ID,
   CLIENT_SECRET,
   VERSION,
   neighborhood_latitude,
   neighborhood_longitude,
   radius,
   LIMIT)
url

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cb9161f1ed2196d92b9d60f'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c062f964a52011f820e3',
       'name': 'The Big Carrot Natural Food Market',
       'location': {'address': '125 Southwood Dr',
        'lat': 43.678879,
        'lng': -79.297734,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.678879,
          'lng': -79.297734}],
        'distance': 471,
        'postalCode': 'M4E 0B8',
   

In [30]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']
    
 

In [31]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) #flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


,name,categories,lat,lng
0,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
1,Grover Pub and Grub,Pub,43.679181,-79.297215
2,Starbucks,Coffee Shop,43.678798,-79.298045
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [32]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.
